In [11]:
import numpy as np
import pandas as pd

In [12]:
#   Tree cơ bản 
class SimpleRegressionTree:
    def __init__(self, max_depth=2, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None  

    def _mse(self, y):
        if len(y) == 0:
            return 0
        mean_y = np.mean(y)
        return np.mean((y - mean_y) ** 2)

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        best_mse = float('inf')
        best_feature = None
        best_threshold = None
        best_left_idx = None
        best_right_idx = None

        for feature in range(n_features):
            # Lấy giá trị feature và sort
            vals = X[:, feature]
            sorted_idx = np.argsort(vals)
            sorted_X = vals[sorted_idx]
            sorted_y = y[sorted_idx]

            # Thử các ngưỡng giữa các giá trị khác nhau
            for i in range(1, n_samples):
                if sorted_X[i] == sorted_X[i-1]:
                    continue
                threshold = (sorted_X[i] + sorted_X[i-1]) / 2

                left_idx = sorted_idx[:i]
                right_idx = sorted_idx[i:]

                if len(left_idx) < self.min_samples_split or len(right_idx) < self.min_samples_split:
                    continue

                mse_left = self._mse(sorted_y[:i])
                mse_right = self._mse(sorted_y[i:])
                mse_split = (len(left_idx) * mse_left + len(right_idx) * mse_right) / n_samples

                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature = feature
                    best_threshold = threshold
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        return best_feature, best_threshold, best_left_idx, best_right_idx

    def _build_tree(self, X, y, depth=0):
        if len(y) < self.min_samples_split or depth >= self.max_depth:
            return {"value": np.mean(y)}  # lá: trung bình y

        feature, threshold, left_idx, right_idx = self._find_best_split(X, y)

        if feature is None:
            return {"value": np.mean(y)}

        left_tree = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_tree = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return {
            "feature": feature,
            "threshold": threshold,
            "left": left_tree,
            "right": right_tree
        }

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        return self

    def _predict_one(self, x, node):
        if "value" in node:
            return node["value"]
        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(x, node["left"])
        else:
            return self._predict_one(x, node["right"])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])

In [13]:
#   Gradient Boosting Regression
class GradientBoostingRegressor:
    def __init__(self, n_estimators=30, learning_rate=0.1, max_depth=2, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
        self.initial_prediction = None
        self.trees = []
        self.tree_weights = []

    def fit(self, X, y):
        self.initial_prediction = np.mean(y)
        predictions = np.full_like(y, self.initial_prediction, dtype=float)
        
        print(f"Dự đoán ban đầu (mean của y): {self.initial_prediction:.2f}\n")

        for i in range(self.n_estimators):
            residuals = y - predictions

            tree = SimpleRegressionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X, residuals)

            update = tree.predict(X)

            predictions += self.learning_rate * update

            self.trees.append(tree)
            self.tree_weights.append(self.learning_rate)

            mse = np.mean((y - predictions) ** 2)
            mean_abs_res = np.mean(np.abs(residuals))
            print(f"Cây {i+1:2d} | MSE: {mse:.4f} | Mean |residual|: {mean_abs_res:.4f}")

        print("\nHuấn luyện hoàn tất.")
        return self

    def predict(self, X):
        pred = np.full(X.shape[0], self.initial_prediction, dtype=float)
        for tree, weight in zip(self.trees, self.tree_weights):
            pred += weight * tree.predict(X)
        return pred

In [ ]:
#   ĐỌC DỮ LIỆU TỪ FILE CSV
color_map = {'Hồng': 0, 'Xanh': 1, 'Đỏ': 2}
gender_map = {'Nam': 0, 'Nữ': 1}

def load_data(csv_path):
    df = pd.read_csv(csv_path)
    
    df['Màu_orig'] = df['Màu']          
    df['Giới tính_orig'] = df['Giới tính']  
    

    df['Màu'] = df['Màu'].map(color_map)
    df['Giới tính'] = df['Giới tính'].map(gender_map)
    

    if df['Màu'].isnull().any() or df['Giới tính'].isnull().any():
        print("CẢNH BÁO: Một số giá trị trong cột 'Màu' hoặc 'Giới tính' không khớp với mapping!")
        print("Các giá trị duy nhất trong 'Màu':", df['Màu_orig'].unique())
        print("Các giá trị duy nhất trong 'Giới tính':", df['Giới tính_orig'].unique())
    

    X = df[['Chiều cao', 'Màu', 'Giới tính']].values.astype(float)
    y = df['Cân nặng'].values.astype(float)
    
    return X, y, df

In [ ]:
#   CHẠY CHƯƠNG TRÌNH
print("Đang đọc dữ liệu huấn luyện từ train.csv...")
X_train, y_train, df_train = load_data('train.csv')

print("\nDữ liệu huấn luyện (5 mẫu đầu):")
print(df_train.head())

gb = GradientBoostingRegressor(
    n_estimators=40,         
    learning_rate=0.1,
    max_depth=2,
    min_samples_split=2
)

gb.fit(X_train, y_train)

y_pred_train = gb.predict(X_train)

print("\nKết quả trên tập huấn luyện:")
print("  Chiều cao | Màu | Giới | Thực tế | Dự đoán | Residual")
for i in range(len(y_train)):
    row = df_train.iloc[i]
    print(f"  {row['Chiều cao']:.2f}      | {int(row['Màu']):3d} | {int(row['Giới tính']):4d} | "
          f"{y_train[i]:7.0f} | {y_pred_train[i]:7.2f} | {y_train[i] - y_pred_train[i]:8.2f}")

Đang đọc dữ liệu huấn luyện từ train.csv...

Dữ liệu huấn luyện (5 mẫu đầu):
   Chiều cao  Màu  Giới tính  Cân nặng Màu_orig Giới tính_orig
0        1.6    0          0        88     Hồng            Nam
1        1.6    1          1        76     Xanh             Nữ
2        1.5    0          1        56     Hồng             Nữ
3        1.8    2          0        73       Đỏ            Nam
4        1.5    0          0        77     Hồng            Nam
Dự đoán ban đầu (mean của y): 73.00

Cây  1 | MSE: 98.5237 | Mean |residual|: 8.5000
Cây  2 | MSE: 83.1530 | Mean |residual|: 7.8500
Cây  3 | MSE: 72.2854 | Mean |residual|: 7.2650
Cây  4 | MSE: 62.6445 | Mean |residual|: 6.7886
Cây  5 | MSE: 52.6736 | Mean |residual|: 6.4278
Cây  6 | MSE: 44.5971 | Mean |residual|: 5.9481
Cây  7 | MSE: 39.0594 | Mean |residual|: 5.5164
Cây  8 | MSE: 34.1341 | Mean |residual|: 5.1754
Cây  9 | MSE: 28.8250 | Mean |residual|: 4.9141
Cây 10 | MSE: 24.5247 | Mean |residual|: 4.5590
Cây 11 | MSE: 21.6806 | Mean

In [16]:
#   ĐÁNH GIÁ TRÊN TEST SET
print("\n" + "="*60)
print("ĐANG ĐỌC VÀ DỰ ĐOÁN TRÊN TEST SET (test.csv)")
print("="*60)

X_test, y_test, df_test = load_data('test.csv')

y_pred_test = gb.predict(X_test)

print("\nKết quả trên tập kiểm tra:")
print("  Chiều cao | Màu   | Giới | Thực tế | Dự đoán | Residual")
for i in range(len(y_test)):
    row = df_test.iloc[i]
    print(f"  {row['Chiều cao']:.2f}      | {row['Màu_orig']:5} | {row['Giới tính_orig']:4} | "
          f"{y_test[i]:7.0f} | {y_pred_test[i]:7.2f} | {y_test[i] - y_pred_test[i]:8.2f}")

print(f"\nMSE trên test: {np.mean((y_test - y_pred_test)**2):.4f}")


ĐANG ĐỌC VÀ DỰ ĐOÁN TRÊN TEST SET (test.csv)

Kết quả trên tập kiểm tra:
  Chiều cao | Màu   | Giới | Thực tế | Dự đoán | Residual
  1.70      | Hồng  | Nam  |      84 |   83.71 |     0.29
  1.55      | Xanh  | Nữ   |      70 |   71.96 |    -1.96
  1.65      | Đỏ    | Nam  |      80 |   79.51 |     0.49
  1.45      | Hồng  | Nữ   |      62 |   58.00 |     4.00
  1.75      | Xanh  | Nam  |      88 |   75.04 |    12.96

MSE trên test: 37.6209
